In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import model_selection
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
import warnings
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Input, Dropout,BatchNormalization
from scikeras.wrappers import KerasClassifier
import random
from tensorflow.keras import backend
random.seed(1)

In [10]:
Data = pd.read_csv('datasets/Data.csv')

In [11]:
Data.shape


(19158, 14)

In [12]:
Data.head()

,Enrollee_id,City,City_development_index,Gender,Relevent_experience,Enrolled_university,Education_level,Major_discipline,Experience,Company_size,Company_type,Last_new_job,Training_hours,Target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [13]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Enrollee_id             19158 non-null  int64  
 1   City                    19158 non-null  object 
 2   City_development_index  19158 non-null  float64
 3   Gender                  14650 non-null  object 
 4   Relevent_experience     19158 non-null  object 
 5   Enrolled_university     18772 non-null  object 
 6   Education_level         18698 non-null  object 
 7   Major_discipline        16345 non-null  object 
 8   Experience              19093 non-null  object 
 9   Company_size            13220 non-null  object 
 10  Company_type            13018 non-null  object 
 11  Last_new_job            18735 non-null  object 
 12  Training_hours          19158 non-null  int64  
 13  Target                  19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [14]:
Data.duplicated().sum()

0

In [15]:
Data.isnull().sum()

Enrollee_id                  0
City                         0
City_development_index       0
Gender                    4508
Relevent_experience          0
Enrolled_university        386
Education_level            460
Major_discipline          2813
Experience                  65
Company_size              5938
Company_type              6140
Last_new_job               423
Training_hours               0
Target                       0
dtype: int64

In [16]:
round(Data.isnull().sum()/Data.isnull().count() * 100, 2)

Enrollee_id                0.00
City                       0.00
City_development_index     0.00
Gender                    23.53
Relevent_experience        0.00
Enrolled_university        2.01
Education_level            2.40
Major_discipline          14.68
Experience                 0.34
Company_size              30.99
Company_type              32.05
Last_new_job               2.21
Training_hours             0.00
Target                     0.00
dtype: float64

In [17]:
Data['Target'].value_counts(1)

Target
0    0.750652
1    0.249348
Name: proportion, dtype: float64

In [18]:
Data.describe().T

,count,mean,std,min,25%,50%,75%,max
Enrollee_id,19158.0,16875.358179,9616.292592,1.000,8554.25,16982.500,25169.75,33380.000
City_development_index,19158.0,0.828848,0.123362,0.448,0.74,0.903,0.92,0.949
Training_hours,19158.0,65.366896,60.058462,1.000,23.00,47.000,88.00,336.000
Target,19158.0,0.249348,0.432647,0.000,0.00,0.000,0.00,1.000


In [19]:
Data.nunique()

Enrollee_id               19158
City                        123
City_development_index       93
Gender                        3
Relevent_experience           2
Enrolled_university           3
Education_level               5
Major_discipline              6
Experience                   22
Company_size                  8
Company_type                  6
Last_new_job                  6
Training_hours              241
Target                        2
dtype: int64

In [20]:
for i in Data.describe(include = ["object"]).columns:
    print("Unique values in", i, "are :")
    print(Data[i].value_counts())
    print("*" * 50)

Unique values in City are :
City
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: count, Length: 123, dtype: int64
**************************************************
Unique values in Gender are :
Gender
Male      13221
Female     1238
Other       191
Name: count, dtype: int64
**************************************************
Unique values in Relevent_experience are :
Relevent_experience
Has relevent experience    13792
No relevent experience      5366
Name: count, dtype: int64
**************************************************
Unique values in Enrolled_university are :
Enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: count, dtype: int64
**************************************************
Unique values in Education_level are :
Education_level
Graduate          11598
Masters            4361
H

In [24]:
Data.drop(columns='Enrollee_id',inplace=True)

In [25]:
!pip install dask-ml


   ---------------------------------------- 0.0/149.8 kB ? eta -:--:--
   ---------------------------------------- 149.8/149.8 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ------------- -------------------------- 0.4/1.2 MB 12.5 MB/s eta 0:00:01
   -------------------------------- ------- 1.0/1.2 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------  1.2/1.2 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 8.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------- ------------------------------- 0.2/1.0 MB 6.7 MB/s eta 0:00:01
   ----------------------------- ---------- 0.7/1.0 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------  1.0/1.0 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ------- --------------------


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: C:\Users\Sandeep\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [26]:
!pip install keras-tuner

   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   -------------------------------------- - 122.9/129.1 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 129.1/129.1 kB 3.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: C:\Users\Sandeep\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: C:\Users\Sandeep\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip
